# MeteoBlue Weather data



## Importing the relevant modules


In [1]:
%matplotlib inline
import sys, os
import matplotlib
matplotlib.style.use("ggplot")
import matplotlib.pyplot as plt
import pandas as pd

data_dir = os.path.join(os.getcwd(), "data")
import pcse
print("This notebook was built with:")
print("python version: %s " % sys.version)
print("PCSE version: %s" %  pcse.__version__)

This notebook was built with:
python version: 3.9.7 (tags/v3.9.7:1016ef3, Aug 30 2021, 20:19:38) [MSC v.1929 64 bit (AMD64)] 
PCSE version: 5.4.2


In [3]:
import requests # for making requests to an API

import uuid
import csv
from csv import DictWriter
from csv import writer

#declare the key 
#start date and end date is got from the agro weather of the CROP
key="Place_key_here"
my_start_date="2020-01-01"
my_end_date="2020-12-31"
mylat=1.070
mylon=34.470


#declare metadata for your file

country="Kenya"
station="Migori"
description = 'Observed data from Migori Station in Kenya'
source      = 'meteoblue histortbasic and historysolar'
contact     = 'KALRO/agriBORA'
elevation = 1500
angstromA = 0.18 
angstromB = 0.55 
hasSunshine = False

#generate unique id
uuidChar=uuid.uuid1()

#declared the file name and path
filename=f"{station}-{uuidChar}"
filename_with_path=f"data/meteo/{filename}.csv"

In [4]:
print(f"File name to be generated: {filename_with_path}")

File name to be generated: data/meteo/Migori-82a40d48-2b2d-11ec-a635-00155da43cee.csv


In [5]:
#declare the class to call data
class CustomMeteoblue:
    #fetch weather data
    def weatherData(lat=mylat,lon=mylon,start_date=my_start_date,end_date=my_end_date,api_key=key):

        url=f"http://my.meteoblue.com/packages/historybasic-day?lat={lat}&lon={lon}&startdate={start_date}&enddate={end_date}&apikey={api_key}"

        r= requests.get(url)

        return r.json()
    
    #fetch radiance
    def radianceData(lat=mylat,lon=mylon,start_date=my_start_date,end_date=my_end_date,api_key=key):

        url=f"http://my.meteoblue.com/packages/historysolar-day?lat={lat}&lon={lon}&startdate={start_date}&enddate={end_date}&apikey={api_key}"

        r= requests.get(url)

        return r.json()


    #Fetch historic agro data
    def historicAgroData(lat=mylat,lon=mylon,start_date=my_start_date,end_date=my_end_date,api_key=key):

        url=f"http://my.meteoblue.com/packages/historyagro-day?lat={lat}&lon={lon}&startdate={start_date}&enddate={end_date}&apikey={api_key}"

        r= requests.get(url)

        return r.json()

In [6]:
weather_data=CustomMeteoblue.weatherData()
radiance_data=CustomMeteoblue.radianceData()
historic_agro_data=CustomMeteoblue.historicAgroData()

# Shows how the response looks like

# end_data={'metadata': {'name': '', 'latitude': 47.56, 'longitude': 7.57, 'height': 282, 'timezone_abbrevation': 'CEST', 'utc_timeoffset': 2.0, 'modelrun_utc': '4716-04-07 12:00'}, 'units': {'time': 'YYYY-MM-DD hh:mm', 'pressure': 'hPa', 'relativehumidity': 'percent', 'temperature': 'C', 'precipitation': 'mm', 'winddirection': 'degree', 'windspeed': 'ms-1'}, 'history_day': {'time': ['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04', '2020-01-05', '2020-01-06', '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-11', '2020-01-12', '2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-18', '2020-01-19', '2020-01-20', '2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31'], 'sealevelpressure_max': [1033, 1032, 1028, 1035, 1035, 1033, 1032, 1031, 1028, 1031, 1037, 1033, 1030, 1019, 1026, 1030, 1026, 1031, 1041, 1043, 1043, 1038, 1035, 1028, 1024, 1022, 1020, 1009, 1019, 1019, 1016], 'sealevelpressure_min': [1031, 1027, 1025, 1027, 1034, 1026, 1026, 1029, 1018, 1017, 1031, 1029, 1019, 1017, 1019, 1026, 1020, 1026, 1030, 1041, 1038, 1033, 1027, 1022, 1020, 1019, 1008, 1002, 1007, 1012, 1013], 'sealevelpressure_mean': [1032, 1030, 1026, 1031, 1034, 1029, 1029, 1030, 1023, 1023, 1034, 1031, 1024, 1018, 1021, 1028, 1023, 1029, 1035, 1041, 1041, 1036, 1030, 1026, 1022, 1020, 1014, 1005, 1013, 1015, 1014], 'temperature_max': [6.94, 8.07, 11.91, 8.14, 7.08, 8.06, 7.67, 8.73, 13.18, 10.99, 8.26, 7.62, 10.27, 13.39, 13.28, 13.03, 9.14, 6.61, 5.0, 6.08, 5.59, 5.63, 4.37, 10.79, 10.75, 11.3, 12.73, 8.26, 7.4, 14.07, 15.42], 'temperature_min': [-0.33, 1.54, 2.02, 2.44, 0.61, -0.85, 2.72, 1.44, 8.93, 5.52, 3.16, 1.38, 3.24, 5.52, 4.54, 4.93, 3.75, 2.04, -0.56, -1.93, -1.67, -2.04, -1.42, 1.67, 5.25, 3.22, 5.3, 3.87, 2.79, 2.05, 9.18], 'temperature_mean': [2.86, 4.22, 6.55, 6.18, 3.39, 2.79, 5.41, 5.32, 10.36, 8.33, 5.55, 3.78, 6.17, 8.61, 7.62, 7.93, 5.92, 4.02, 1.27, 1.66, 0.94, 0.74, 0.88, 6.37, 8.09, 6.74, 8.33, 6.24, 5.05, 7.41, 12.11], 'precipitation': [0.0, 0.0, 0.4, 2.6, 0.0, 0.0, 0.2, 0.1, 0.1, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.7, 0.9, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.4, 3.7, 3.3, 0.9, 0.0], 'snowfraction': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.17, 0.04, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'winddirection': [104, 119, 243, 285, 102, 123, 257, 115, 150, 248, 358, 141, 125, 170, 166, 125, 211, 264, 4, 70, 79, 16, 8, 126, 112, 154, 212, 253, 271, 210, 250], 'windspeed_max': [2.89, 2.47, 6.11, 4.66, 3.61, 3.44, 4.73, 2.52, 4.03, 8.26, 3.5, 2.05, 3.11, 4.52, 4.2, 3.53, 7.81, 8.83, 4.44, 7.43, 4.22, 2.28, 1.8, 2.38, 2.21, 3.33, 9.69, 10.82, 9.6, 7.07, 7.37], 'windspeed_min': [0.1, 0.89, 0.2, 2.31, 0.22, 1.08, 0.78, 0.32, 0.42, 1.78, 0.5, 0.1, 0.94, 1.12, 0.5, 1.2, 0.6, 0.81, 0.14, 0.61, 0.32, 0.63, 0.1, 0.28, 0.73, 0.85, 1.3, 7.37, 3.86, 2.0, 2.47], 'windspeed_mean': [1.28, 1.65, 3.67, 3.83, 1.81, 2.34, 2.5, 1.9, 2.09, 5.35, 1.77, 1.12, 1.95, 2.34, 2.5, 2.59, 2.97, 4.31, 1.63, 3.48, 1.72, 1.34, 0.74, 1.45, 1.45, 1.8, 4.03, 9.4, 6.92, 3.75, 4.97], 'relativehumidity_max': [89, 89, 88, 95, 91, 88, 87, 95, 84, 87, 90, 86, 89, 67, 67, 82, 96, 88, 96, 95, 89, 91, 93, 78, 86, 85, 81, 84, 86, 87, 82], 'relativehumidity_min': [64, 60, 56, 59, 64, 59, 64, 61, 35, 41, 63, 63, 42, 45, 45, 53, 59, 62, 65, 51, 52, 65, 60, 62, 60, 63, 49, 66, 66, 44, 66], 'relativehumidity_mean': [77, 78, 76, 80, 81, 75, 76, 82, 68, 67, 80, 76, 60, 53, 58, 70, 75, 78, 85, 73, 73, 79, 76, 70, 77, 71, 67, 73, 79, 73, 75]}}
# end_data2={'metadata': {'name': '', 'latitude': 47.56, 'longitude': 7.57, 'height': 282, 'timezone_abbrevation': 'CEST', 'utc_timeoffset': 2.0, 'modelrun_utc': '4716-04-07 12:00'}, 'units': {'time': 'YYYY-MM-DD hh:mm', 'radiation': 'Wm-2'}, 'history_day': {'time': ['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04', '2020-01-05', '2020-01-06', '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-11', '2020-01-12', '2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-18', '2020-01-19', '2020-01-20', '2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31'], 'ghi_sum': [1805.81, 1731.94, 1475.62, 930.05, 1480.96, 1828.95, 1227.31, 1146.32, 1363.48, 907.8, 1179.25, 1849.42, 1557.5, 1759.53, 2043.44, 1958.0, 1357.25, 954.08, 1167.68, 1955.33, 2201.86, 2240.13, 2305.1, 2267.72, 2038.1, 1857.43, 1719.48, 909.58, 993.24, 2127.99, 591.85], 'dif_sum': [1176.37, 1071.69, 778.3, 374.02, 763.94, 1148.88, 501.08, 428.83, 728.5, 432.26, 461.6, 1088.02, 771.76, 967.37, 1278.31, 1152.15, 748.63, 399.03, 353.88, 1091.23, 1337.34, 1359.66, 1415.62, 1343.77, 1075.38, 859.99, 856.96, 529.4, 534.52, 1068.7, 514.96], 'dni_sum': [2290.13, 2428.43, 2659.11, 2150.69, 2698.33, 2459.61, 2747.47, 2719.62, 2277.57, 1775.25, 2640.66, 2710.48, 2819.22, 2724.38, 2606.65, 2788.42, 2022.85, 2081.07, 2893.08, 2926.57, 2809.81, 2823.74, 2808.34, 2914.91, 3083.83, 3172.87, 2700.31, 1455.23, 1717.7, 3215.97, 403.96], 'gni_sum': [3307.53, 3375.85, 3418.72, 2572.98, 3446.96, 3469.98, 3313.25, 3231.01, 2979.15, 2228.84, 3171.71, 3699.5, 3591.17, 3637.25, 3730.61, 3827.43, 2732.02, 2535.17, 3366.13, 3944.87, 4006.92, 4039.99, 4067.03, 4127.59, 4116.21, 4060.18, 3550.0, 1968.93, 2259.11, 4273.0, 806.06], 'extraterrestrialradiation_sum': [2551.36, 2567.62, 2585.05, 2603.62, 2623.3, 2644.16, 2666.2, 2689.48, 2714.63, 2741.12, 2769.1, 2798.56, 2829.34, 2861.35, 2894.8, 2929.66, 2966.14, 3003.68, 3042.29, 3082.02, 3122.97, 3165.14, 3208.82, 3253.99, 3300.14, 3347.5, 3396.26, 3445.93, 3496.46, 3548.81, 3602.1]}}

# historic_agro_data={'metadata': {'name': '', 'latitude': 47.56, 'longitude': 7.57, 'height': 282, 'timezone_abbrevation': 'CEST', 'utc_timeoffset': 2.0, 'modelrun_utc': '4716-04-07 12:00'}, 'units': {'time': 'YYYY-MM-DD hh:mm', 'soilmoisture': 'volumetric percent', 'temperature': 'C', 'transpiration': 'mm'}, 'history_day': {'time': ['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04', '2020-01-05', '2020-01-06', '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-11', '2020-01-12', '2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-18', '2020-01-19', '2020-01-20', '2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31'], 'soiltemperature_0to10cm_max': [5.63, 6.44, 9.28, 8.7, 6.79, 6.53, 8.17, 7.63, 11.78, 11.07, 8.93, 7.33, 9.47, 10.73, 10.11, 11.71, 7.28, 7.96, 5.91, 5.48, 4.84, 5.15, 4.25, 8.68, 9.16, 10.83, 10.77, 7.4, 7.79, 11.11, 14.23], 'soiltemperature_0to10cm_min': [1.07, 0.85, 1.73, 3.14, 1.7, 1.05, 0.98, 1.19, 5.98, 5.15, 2.94, 1.21, 2.02, 1.53, 1.25, 3.46, 1.18, 1.81, 1.19, 0.61, 0.35, 0.34, 0.41, 0.54, 3.45, 0.91, 3.78, 3.71, 3.17, 1.58, 7.66], 'soiltemperature_0to10cm_mean': [2.22, 2.41, 4.82, 6.31, 3.75, 2.49, 3.8, 4.11, 7.76, 7.55, 6.14, 3.06, 5.1, 5.4, 5.15, 6.53, 4.46, 4.23, 2.56, 2.15, 1.75, 1.82, 1.57, 3.31, 5.64, 4.92, 6.95, 6.06, 4.91, 5.85, 10.17], 'soilmoisture_0to10cm_max': [0.28, 0.28, 0.28, 0.29, 0.28, 0.28, 0.28, 0.27, 0.27, 0.27, 0.27, 0.27, 0.27, 0.27, 0.27, 0.26, 0.32, 0.3, 0.28, 0.27, 0.27, 0.27, 0.27, 0.27, 0.27, 0.27, 0.27, 0.31, 0.3, 0.29, 0.3], 'soilmoisture_0to10cm_min': [0.28, 0.28, 0.28, 0.28, 0.28, 0.27, 0.27, 0.27, 0.27, 0.26, 0.27, 0.27, 0.27, 0.26, 0.26, 0.26, 0.26, 0.28, 0.27, 0.27, 0.27, 0.27, 0.27, 0.27, 0.27, 0.26, 0.26, 0.26, 0.29, 0.28, 0.28], 'soilmoisture_0to10cm_mean': [0.28, 0.28, 0.28, 0.28, 0.28, 0.27, 0.27, 0.27, 0.27, 0.27, 0.27, 0.27, 0.27, 0.27, 0.26, 0.26, 0.28, 0.29, 0.28, 0.27, 0.27, 0.27, 0.27, 0.27, 0.27, 0.27, 0.26, 0.28, 0.29, 0.29, 0.29], 'skintemperature_max': [7.48, 8.33, 11.99, 9.95, 8.33, 8.33, 9.76, 10.29, 14.32, 12.79, 10.4, 8.95, 11.48, 13.47, 12.5, 13.96, 9.01, 9.19, 7.47, 7.11, 6.53, 6.82, 5.88, 11.06, 13.1, 13.59, 13.55, 7.97, 9.27, 14.0, 16.65], 'skintemperature_min': [-0.69, -0.56, 0.99, 1.51, 0.05, -0.95, -0.31, -0.31, 5.24, 4.21, 0.96, -0.6, 0.53, 0.16, -0.33, 2.21, -0.28, 0.0, -0.55, -1.16, -1.54, -1.55, -1.32, -0.62, 1.92, -0.45, 2.67, 2.55, 2.41, 0.02, 7.29], 'skintemperature_mean': [1.62, 2.11, 5.44, 6.14, 3.47, 2.0, 3.76, 4.49, 8.33, 7.81, 5.76, 2.56, 5.32, 5.81, 5.09, 6.6, 4.14, 3.52, 1.81, 1.52, 1.12, 1.17, 1.06, 3.77, 6.19, 5.3, 7.32, 5.66, 4.74, 6.47, 10.95], 'evapotranspiration': [0.21, 0.15, 0.42, 0.59, 0.3, 0.27, 0.33, 0.22, 0.37, 0.86, 0.41, 0.2, 0.39, 0.46, 0.41, 0.42, 0.28, 0.67, 0.26, 0.49, 0.37, 0.27, 0.27, 0.19, 0.29, 0.43, 0.76, 1.44, 0.95, 0.72, 0.92], 'referenceevapotranspiration_fao': [0.69, 1.24, 1.53, 1.04, 0.73, 1.34, 1.24, 0.89, 1.44, 2.06, 1.25, 0.89, 1.23, 2.24, 2.19, 1.36, 0.8, 1.37, 0.93, 1.15, 0.94, 1.27, 1.32, 1.2, 1.11, 1.75, 2.14, 1.65, 1.06, 2.05, 2.33], 'dewpointtemperature_max': [1.22, 2.6, 6.44, 5.94, 1.03, 1.92, 3.17, 6.39, 8.01, 5.29, 4.12, 2.62, 2.86, 3.41, 2.79, 4.48, 6.55, 2.27, 0.02, -2.53, -1.03, -0.16, -2.07, 4.95, 5.53, 4.82, 3.61, 3.27, 2.63, 7.38, 9.39], 'dewpointtemperature_min': [-3.0, -2.6, -1.21, 0.02, -1.04, -4.23, -2.29, 0.46, -4.49, -3.3, 0.82, -4.15, -5.76, -4.51, -3.67, 1.66, -2.14, -0.29, -2.6, -3.83, -5.61, -4.05, -4.24, -3.41, 2.21, -1.27, 1.11, 0.28, 0.59, -1.5, 6.3], 'dewpointtemperature_mean': [-0.82, 0.62, 2.51, 2.86, 0.41, -1.37, 1.51, 2.46, 4.27, 2.32, 2.41, -0.08, -1.05, -0.46, -0.19, 2.67, 1.75, 0.51, -1.05, -2.97, -3.48, -2.48, -3.01, 1.26, 4.19, 1.89, 2.49, 1.72, 1.73, 2.56, 7.85], 'wetbulbtemperature_max': [3.85, 4.98, 8.03, 6.51, 3.72, 4.5, 4.85, 7.18, 9.62, 6.82, 4.74, 4.74, 6.26, 7.9, 7.38, 8.02, 6.69, 3.38, 1.98, 1.8, 1.55, 2.56, 1.1, 7.4, 7.65, 7.57, 7.51, 5.14, 4.57, 8.84, 11.67], 'wetbulbtemperature_min': [-1.57, 0.39, 0.89, 0.98, -0.5, -2.36, 0.22, 0.8, 3.51, 2.86, 1.67, -0.95, -0.11, 1.39, 0.75, 2.97, 0.91, 0.71, -1.53, -2.47, -3.29, -3.18, -2.49, -0.87, 3.79, 0.94, 3.27, 1.82, 1.4, 0.39, 7.32], 'wetbulbtemperature_mean': [0.8, 2.17, 4.21, 4.23, 1.64, 0.54, 3.17, 3.59, 7.09, 5.08, 3.67, 1.62, 2.61, 4.13, 3.66, 4.95, 3.59, 2.02, -0.1, -0.67, -1.31, -1.06, -1.15, 3.54, 5.76, 3.99, 5.1, 3.66, 3.07, 4.67, 9.43]}}



In [7]:
#get data from the Response by KEYS

time=weather_data["history_day"]["time"]
sealevelpressure_max=weather_data["history_day"]["sealevelpressure_max"]
sealevelpressure_min=weather_data["history_day"]["sealevelpressure_min"]
sealevelpressure_mean=weather_data["history_day"]["sealevelpressure_mean"]

temperature_max=weather_data["history_day"]["temperature_max"]
temperature_min=weather_data["history_day"]["temperature_min"]
temperature_mean=weather_data["history_day"]["temperature_mean"]

precipitation=weather_data["history_day"]["precipitation"]
snowfraction=weather_data["history_day"]["snowfraction"]

winddirection=weather_data["history_day"]["winddirection"]
windspeed_max=weather_data["history_day"]["windspeed_max"]
windspeed_min=weather_data["history_day"]["windspeed_min"]
windspeed_mean=weather_data["history_day"]["windspeed_mean"]
relativehumidity_max=weather_data["history_day"]["relativehumidity_max"]
relativehumidity_min=weather_data["history_day"]["relativehumidity_min"]
relativehumidity_mean=weather_data["history_day"]["relativehumidity_mean"]

evapotrnspiration_data=historic_agro_data["history_day"]["evapotranspiration"]

extraterrestrialradiation_sum=radiance_data["history_day"]["extraterrestrialradiation_sum"]
ghi_sum=radiance_data["history_day"]["ghi_sum"]


In [8]:

headersCSV = ['DAY','IRRAD','TMIN','TMAX','VAP','WIND','RAIN','SNOWDEPTH']  

# The data assigned to the dictionary 
dict={'DAY':time[0],'IRRAD':ghi_sum[0],'TMIN':temperature_min[0],'TMAX':temperature_max[0],'VAP':'aa','WIND':windspeed_max[0],'RAIN':precipitation[0],'SNOWDEPTH':snowfraction[0]}
  


    

def appendWeatherDataToCSV(dict=dict):
    with open(filename_with_path, 'a', newline='') as f_object:
        # Pass the CSV  file object to the Dictwriter() function
        # Result - a DictWriter object
        dictwriter_object = DictWriter(f_object,delimiter=",", fieldnames=headersCSV)
        # Pass the data in the dictionary as an argument into the writerow() function
        dictwriter_object.writerow(dict)
        # Close the file object
        f_object.close()

        
def appendTextHeader(dict=dict):
    with open(filename_with_path, 'a', newline='') as f_object:
        writer=csv.writer(f_object)
      
        writer.writerow(dict)
        # Close the file object
        f_object.close()
        
        
test_list=['## Site Characteristic']
test_lis2=[f"Country     = '{country}'"]
test_list3=[f"Station     = '{station}'"]
test_list4=[f"Description = '{description}'"]

test_list5=[f"Source      = '{source}'"]
test_list6=[f"Contact     = '{contact}'"]
test_list7=[f"Longitude = {mylon}; Latitude = {mylat}; Elevation = {elevation}; AngstromA = {angstromA}; AngstromB = {angstromB}; HasSunshine = {hasSunshine}"]
test_list8=[f"## Daily weather observations {my_start_date} - {my_end_date}"]




test_lists=[
   test_list, test_lis2,test_list3,test_list4,test_list5,test_list6,test_list7,test_list8
]





def writeTextHeader():
    i=0

    while i < len(test_lists):
        appendTextHeader(test_lists[i])
        print(f" finished list {test_lists[i]}")

        i+=1


    
def writeCSVData():
    i=0
    
    while i < len(time):

        mydict={'DAY':time[i],'IRRAD':ghi_sum[i],'TMIN':temperature_min[i],'TMAX':temperature_max[i],'VAP':evapotrnspiration_data[i],'WIND':windspeed_max[i],'RAIN':precipitation[i],'SNOWDEPTH':snowfraction[i]}
        appendWeatherDataToCSV(dict=mydict)
        i+=1   

def runbuild():
    writeTextHeader()
    appendTextHeader(headersCSV)
    writeCSVData()
    


        

runbuild()






 finished list ['## Site Characteristic']
 finished list ["Country     = 'Kenya'"]
 finished list ["Station     = 'Migori'"]
 finished list ["Description = 'Observed data from Migori Station in Kenya'"]
 finished list ["Source      = 'meteoblue histortbasic and historysolar'"]
 finished list ["Contact     = 'KALRO/agriBORA'"]
 finished list ['Longitude = 34.47; Latitude = 1.07; Elevation = 1500; AngstromA = 0.18; AngstromB = 0.55; HasSunshine = False']
 finished list ['## Daily weather observations 2021-03-25 - 2021-08-25']


## Daily weather observations
Daily weather variables are needed for running the simulation. There are several data providers in PCSE for reading weather data, see the section on [weather data providers](http://pcse.readthedocs.io/en/stable/reference_guide.html#weather-data-providers) to get an overview.

For this example we will use weather data from an excel file which provides daily weather data for Wageningen for the period 2004 to 2008. We will read the data from the file using the ExcelWeatherDataProvider:

In [14]:
filename

'Migori-82a40d48-2b2d-11ec-a635-00155da43cee'

In [15]:
from pcse.fileinput import CSVWeatherDataProvider

weatherfile2 = os.path.join(data_dir, 'meteo', f"{filename}.csv")
wdp = CSVWeatherDataProvider(weatherfile2, dateformat="%Y-%m-%d")

print(filename_with_path)
print(wdp)

data/meteo/Migori-82a40d48-2b2d-11ec-a635-00155da43cee.csv
Weather data provided by: CSVWeatherDataProvider
--------Description---------
Weather data for:
Country: Kenya
Station: Migori
Description: Observed data from Migori Station in Kenya
Source: meteoblue histortbasic and historysolar
Contact: KALRO/agriBORA
----Site characteristics----
Elevation: 1500.0
Latitude:   1.070
Longitude: 34.470
Data available for 2021-03-25 - 2021-08-25
Number of missing days: 0

